## Python 3.10
Python 3.10 is the recommended Python version for running the WebUI. Paperspace uses Python 3.9 for their containers so you must use a custom container. Luckily, I've created a container for you to use.
First, delete your current notebook and create a new one following these instructions: https://docs.paperspace.com/gradient/notebooks/runtimes/#how-to-specify-a-custom-container
Make sure to use this container image: `cyberes/gradient-base-py3.10`
You can use the block below to test your Python version.

#### Base Definitions

In [1]:
!export PIP_ROOT_USER_ACTION=ignore
%cd /notebooks

import logging
import os
import subprocess
import sys
import time

import urllib.request

from glob import glob
from pathlib import Path
from urllib.parse import urlparse, parse_qs, unquote


logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


def download_civit_model(url: str, output_path: str, token: str):

    CHUNK_SIZE = 1638400
    USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'

    headers = {
        'Authorization': f'Bearer {token}',
        'User-Agent': USER_AGENT,
    }

    # Disable automatic redirect handling
    class NoRedirection(urllib.request.HTTPErrorProcessor):
        def http_response(self, request, response):
            return response
        https_response = http_response

    request = urllib.request.Request(url, headers=headers)
    opener = urllib.request.build_opener(NoRedirection)
    response = opener.open(request)

    if response.status in [301, 302, 303, 307, 308]:
        redirect_url = response.getheader('Location')

        # Extract filename from the redirect URL
        parsed_url = urlparse(redirect_url)
        query_params = parse_qs(parsed_url.query)
        content_disposition = query_params.get('response-content-disposition', [None])[0]

        if content_disposition:
            filename = unquote(content_disposition.split('filename=')[1].strip('"'))
        else:
            raise Exception('Unable to determine filename')

        response = urllib.request.urlopen(redirect_url)
    elif response.status == 404:
        raise Exception('File not found')
    else:
        raise Exception('No redirect found, something went wrong')

    total_size = response.getheader('Content-Length')

    if total_size is not None:
        total_size = int(total_size)

    if (Path(output_path)/filename).is_file():
        logger.warn(f'{Path(output_path)/filename} already exists, skipping download')
        return

    output = Path(output_path)
    output.mkdir(parents=True, exist_ok=True)

    with open(Path(output_path)/filename, 'wb') as f:
        downloaded = 0
        start_time = time.time()

        while True:
            chunk_start_time = time.time()
            buffer = response.read(CHUNK_SIZE)
            chunk_end_time = time.time()

            if not buffer:
                break

            downloaded += len(buffer)
            f.write(buffer)
            chunk_time = chunk_end_time - chunk_start_time

            if chunk_time > 0:
                speed = len(buffer) / chunk_time / (1024 ** 2)  # Speed in MB/s

            if total_size is not None:
                progress = downloaded / total_size
                sys.stdout.write(f'\rDownloading: {filename} [{progress*100:.2f}%] - {speed:.2f} MB/s')
                sys.stdout.flush()

    end_time = time.time()
    time_taken = end_time - start_time
    hours, remainder = divmod(time_taken, 3600)
    minutes, seconds = divmod(remainder, 60)

    if hours > 0:
        time_str = f'{int(hours)}h {int(minutes)}m {int(seconds)}s'
    elif minutes > 0:
        time_str = f'{int(minutes)}m {int(seconds)}s'
    else:
        time_str = f'{int(seconds)}s'

    sys.stdout.write('\n')
    logger.info(f'Download completed. File saved as: {Path(output_path)/filename}')
    logger.info(f'Downloaded in {time_str}')


def create_symlink(source, dest):
    if dest.is_symlink() and not dest.exists():
        logger.debug(f'Symlink broken, removing: {dest}')
        dest.unlink()
    if not dest.exists():
        os.symlink(src, dest)
    logger.info(f'{os.path.realpath(dest)} -> {dest}')

def create_symlink_2(source, dest):
    if os.path.isdir(dest):
        dest = Path(dest, os.path.basename(source))
    if not dest.exists():
        os.symlink(source, dest)
    logger.info(f'{source} -> {Path(dest).absolute()}')


def clone_repo(dest, remote, branch):
    if not (dest / '.git').exists():
        # It's possible that the repo already exists but the repo has not been downloaded.
        # We will init the repo manually.
        !mkdir -p "{dest}"
        %cd "{dest}"
        !git init
        !git remote add origin "{remote}"
        !git fetch origin "{branch}"
        !git checkout -t "origin/{branch}" -f
    else:
        logger.debug(f'{dest} already downloaded, updating...')
        !cd "{dest}" && git pull origin "{branch}" # no % so we don't interfere with the main process


def download_whl(url, binary_name='xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl'):
    tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
    !wget "{url}" -O "{tmp_dir}/{binary_name}"
    return os.path.join(tmp_dir, binary_name)


def delete_broken_symlinks(dir):
    for file in Path(dir).iterdir():
        if file.is_symlink() and not file.exists():
            logger.debug(f'Symlink broken, removing: {file}')
            file.unlink()


def link_ckpts(source_path, webui_sd_model_path):
    # Link .ckpt and .safetensor/.st files (recursive)
    logger.info(f'Linking .ckpt and .safetensor/.safetensors/.st files in {source_path}')
    source_path = Path(source_path)
    for file in [
        p for p in source_path.rglob('*')
        if p.suffix in ['.ckpt', '.safetensor', '.safetensors', '.st']
    ]:
        if Path(file).parent.parts[-1] not in ['hypernetworks', 'vae', 'embeddings'] :
            if not (webui_sd_model_path / file.name):
                logger.debug(f'New model: {file.name}')
            create_symlink_2(file, webui_sd_model_path)
    # Link config yaml files
    logger.info(f'Linking config .yaml files in {source_path}')
    for file in source_path.glob('*.yaml'):
        create_symlink_2(file, webui_sd_model_path)


def install_xformers(install_pip_xformers):
    if install_pip_xformers:
        logger.info('Installing xformers through pip...')
        !pip install --no-dependencies xformers
    else:
        xformers_whl = None
        found_xformers_whls = glob('/notebooks/xformers-*')
        if len(found_xformers_whls) == 1:
            logger.info('Installing xformers using your pre-built wheel...')
            xformers_whl = found_xformers_whls[0]
            delete_whl = False
        elif len(found_xformers_whls) > 1:
            logger.info('Found more than one Xformers wheel in /notebooks so not doing anything!')
        else:
            logger.info('Installing xformers from wheels on Github...')
            delete_whl = True

            # Set up pip packages
            !pip uninstall -y torch torchvision torchaudio protobuf # Remove existing pytorch install.
            # !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 # Install pytorch for cuda 11.3
            s = subprocess.getoutput('nvidia-smi -L')
            logger.info(f'Your SMI: {s}')
            if 'A4000' in s:
                xformers_whl = download_whl('https://github.com/Cyberes/xformers-compiled/raw/main/a4000/xformers-0.0.18%2Bda27862.d20230413-cp39-cp39-linux_x86_64.whl')
            elif 'A5000' in s:
                xformers_whl = download_whl('https://github.com/Cyberes/xformers-compiled/releases/download/A5000-Nov-1-2022/a5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'A6000' in s:
                xformers_whl = download_whl('https://github.com/Cyberes/xformers-compiled/releases/download/A6000-Nov-1-2022/a6000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'P5000' in s:
                xformers_whl = download_whl('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/p5000/xformers-0.0.16%2B6f3c20f.d20230127-cp39-cp39-linux_x86_64.whl')
            elif 'RTX 4000' in s:
                xformers_whl = download_whl('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-4000-Nov-1-2022/rtx4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'RTX 5000' in s:
                xformers_whl = download_whl('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-5000-Nov-1-2022/rtx5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'A100' in s:
                xformers_whl = download_whl('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            elif 'M4000' in s:
                logger.warning('xformers for M4000 hasn\'t been built yet.')
                # xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
            else:
                logger.warning('GPU not matched to xformers binary so a one-size-fits-all binary was installed. If you have any issues, please build xformers using the Tools block below.')
                xformers_whl = download_whl('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/various/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl')
        if xformers_whl:
            !pip uninstall -y xformers
            # We're going to install xformers without installing any of its dependencies since they should already be installed.
            # If you have any issues then replacing --no-dependencies with --force-reinstall
            !pip install --no-dependencies "{xformers_whl}"
            if delete_whl:
                !rm -rf "{xformers_whl}"


py_version = sys.version.split(' ')[0]
if int(py_version.split('.')[1]) < 10:
    logger.info(f'Your Python version is less than 3.10 -> {py_version}')
else:
    logger.info(f'Your Python version is good: {py_version}')

# Other optional settings
# You don't have to change these if you don't want to.
symlink_to_notebooks = True                            # Enables the creation of symlinks back to /notebooks/
activate_xformers = True                               # Enables the xformers optimizations using pre-built wheels.
                                                       # Setting to True will automatically set up your environment/machine for xformers. 
install_pip_xformers = False                           # Install xformers through pip. Probably won't work because it needs Torch 2.0
link_novelai_anime_vae = True                          # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.
activate_deepdanbooru = False                          # Enable and install DeepDanbooru -> https://github.com/KichangKim/DeepDanbooru
activate_medvram = True                                # Enable medvram option.
                                                       # These are model optimizations which will reduce VRAM usage at the expense of some speed.
                                                       # Set to False if you have a lot of VRAM.
disable_pickle_check = False                           # Disable the automatic check for unexpected data in model files.
                                                       # Leave this set to False unless you have a reason to disable the check.
gradio_port = False                                    # Launch Gradio on a specific port. Set to False to let Gradio choose a port.
                                                       # This disables online Gradio app mode and you will only be able to access it on your local network.
gradio_auth = False                                    # Enable gradio_auth and insecure-extension-access option.
                                                       # Set to a username:password (for example: "me:password") to enable.
search_paperspace_datasets = True                      # Enable searching for checkpoints in /datasets to link to the webui
ui_theme = None                                        # Set the WEB UI theme. Values can be None (default) or 'dark'.
insecure_extension_access = False                      # Force enable extensions without a password.
                                                       # If you don't set a password anyone can install and run arbitrary code on your machine!
                                                       # Instead, use gradio_auth which will automatically enable extensions when set.
gradio_queue = False                                   # Uses gradio queue; experimental option; breaks restart UI button.
civitai_token = input('Your CivitAI Token? ')

# Choose where to store your model checkpoints.
model_storage_dir = '/tmp/stable-diffusion-models' # Free Tier
# model_storage_dir = '/storage/models' # Paid Tier

# Optional path settings
repo_storage_dir = '/storage/stable-diffusion'         # Where to store your Stable Diffusion-related files.
export_storage_dir = '/notebooks/exports'              # Where the generated images will be exported to.
pip_cache_dir = None                                   # The installer can cache pip wheels so you don't have to re-download them
                                                       # every time you start the machine. I recommed setting it to '/storage/pip/cache'

repo_storage_dir = Path(repo_storage_dir)
model_storage_dir = Path(model_storage_dir)
stable_diffusion_webui_path = repo_storage_dir / 'stable-diffusion-webui'

!mkdir -p "{stable_diffusion_webui_path / 'outputs'}"
!mkdir -p "{stable_diffusion_webui_path / 'log'}"

symlinks = [
    (stable_diffusion_webui_path, Path('/notebooks/stable-diffusion-webui')),
    (stable_diffusion_webui_path / 'outputs', Path('/notebooks/outputs')),
    # (stable_diffusion_webui_path / 'log', repo_storage_dir / 'stable-diffusion-webui' / 'outputs' / 'log'),
    (Path('/storage'), Path('/notebooks/storage')),
    (model_storage_dir, Path('/notebooks/models')),
]
if symlink_to_notebooks and repo_storage_dir != '/notebooks':
    logger.info('Creating Symlinks...')
    for src, dest in symlinks:
        create_symlink(src, dest)

clone_repo(
    stable_diffusion_webui_path,
    "https://github.com/AUTOMATIC1111/stable-diffusion-webui",
    "master"
)
# ===================================================================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store symlink_to_notebooks model_storage_dir repo_storage_dir export_storage_dir activate_xformers link_novelai_anime_vae activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth search_paperspace_datasets ui_theme insecure_extension_access pip_cache_dir gradio_queue install_pip_xformers

/notebooks
2024-12-22 13:46:28,353 | INFO : Your Python version is good: 3.10.10


Your CivitAI Token?  ce7f051c7d8a244671e1737f840dfb6f


2024-12-22 13:46:57,753 | INFO : Creating Symlinks...
2024-12-22 13:46:57,755 | INFO : /storage/stable-diffusion/stable-diffusion-webui -> /notebooks/stable-diffusion-webui
2024-12-22 13:46:57,759 | INFO : /storage/stable-diffusion/stable-diffusion-webui/outputs -> /notebooks/outputs
2024-12-22 13:46:57,762 | INFO : /storage -> /notebooks/storage
2024-12-22 13:46:57,767 | DEBUG : Symlink broken, removing: /notebooks/models
2024-12-22 13:46:57,772 | INFO : /tmp/stable-diffusion-models -> /notebooks/models
2024-12-22 13:46:57,774 | DEBUG : /storage/stable-diffusion/stable-diffusion-webui already downloaded, updating...
From https://github.com/AUTOMATIC1111/stable-diffusion-webui
 * branch              master     -> FETCH_HEAD
Already up to date.
Stored 'symlink_to_notebooks' (bool)
Stored 'model_storage_dir' (PosixPath)
Stored 'repo_storage_dir' (PosixPath)
Stored 'export_storage_dir' (str)
Stored 'activate_xformers' (bool)
Stored 'link_novelai_anime_vae' (bool)
Stored 'activate_deepdanb

In [ ]:
!export PIP_ROOT_USER_ACTION=ignore
%cd "{stable_diffusion_webui_path}"

!pip install --upgrade pip
!pip install --upgrade wheel setuptools

if activate_xformers:
    install_xformers(install_pip_xformers)

import launch; launch.prepare_environment()

In [7]:
!export PIP_ROOT_USER_ACTION=ignore
%cd "{stable_diffusion_webui_path}"

# Make sure important directories exists
!mkdir -p "{model_storage_dir}/hypernetworks"
!mkdir -p "{model_storage_dir}/vae"
!mkdir -p "{model_storage_dir}/embeddings"
!mkdir -p "{stable_diffusion_webui_path}/models/hypernetworks"
!mkdir -p "{stable_diffusion_webui_path}/models/VAE"
!mkdir -p "{stable_diffusion_webui_path}/models/Lora"
!mkdir -p "{stable_diffusion_webui_path}/log/images"

if not model_storage_dir.exists():
    logger.error(f'Your model storage directory does not exist: {model_storage_dir}')
    sys.exit(1)

embeddings_urls = [
    "https://civitai.com/api/download/models/253081", # Inna Nobody
]
for url in embeddings_urls:
    dest = model_storage_dir/'embeddings'
    logger.debug(f'Downloading Embedding {url} into {dest}')
    download_civit_model(url, dest, civitai_token)

zovya_photoreal = "https://civitai.com/api/download/models/474400"
download_civit_model(zovya_photoreal, model_storage_dir, civitai_token)

analog_madness = "https://civitai.com/api/download/models/261539?type=Model&format=SafeTensor&size=pruned&fp=fp16"
download_civit_model(analog_madness, model_storage_dir, civitai_token)

webui_root_model_path = Path(stable_diffusion_webui_path, 'models')
webui_sd_model_path = Path(webui_root_model_path, 'Stable-diffusion')
webui_hypernetwork_path = Path(webui_root_model_path, 'hypernetworks')
webui_vae_path = Path(webui_root_model_path, 'VAE')
webui_lora_model_path = Path(webui_root_model_path, 'Lora')
webui_embeddings_path = Path(stable_diffusion_webui_path, 'embeddings')

# Check for broken symlinks and remove them
logger.info('Removing broken symlinks...')
delete_broken_symlinks(webui_sd_model_path)
delete_broken_symlinks(webui_hypernetwork_path)
delete_broken_symlinks(webui_vae_path)
delete_broken_symlinks(webui_lora_model_path)
delete_broken_symlinks(webui_embeddings_path)

logger.info('Linking checkpoints...')
link_ckpts(model_storage_dir, webui_sd_model_path)

# Link hypernetworks
logger.info('Linking hypernetworks...')
hypernetwork_source_path = Path(model_storage_dir, 'hypernetworks')
if hypernetwork_source_path.is_dir():
    for file in hypernetwork_source_path.iterdir():
        create_symlink_2(hypernetwork_source_path / file, webui_hypernetwork_path)
else:
    logger.warning(f'Hypernetwork storage directory not found: {hypernetwork_source_path}')

# Link VAEs
logger.info('Linking VAEs...')
vae_source_path = Path(model_storage_dir, 'vae')
if vae_source_path.is_dir():
    for file in vae_source_path.iterdir():
        create_symlink_2(vae_source_path / file, webui_vae_path)
else:
    logger.warning(f'VAE storage directory not found: {vae_source_path}')

# Link Lora
logger.info('Linking Loras...')
lora_source_path = Path(model_storage_dir, 'Lora')
if lora_source_path.is_dir():
    for file in lora_source_path.iterdir():
        create_symlink_2(lora_source_path / file, webui_lora_model_path)
else:
    logger.warning(f'Lora storage directory not found: {lora_source_path}')

# Link Embeddings
logger.info('Linking Embeddings...')
embeddings_source_path = Path(model_storage_dir, 'embeddings')
if embeddings_source_path.is_dir():
    for file in embeddings_source_path.iterdir():
        create_symlink_2(embeddings_source_path / file, webui_embeddings_path)
else:
    logger.warning(f'Embeddings storage directory not found: {embeddings_source_path}')


if search_paperspace_datasets:
    if Path('/datasets').is_dir():
        link_ckpts('/datasets', webui_sd_model_path)
    else:
        logger.warning('No datasets mounted!')

/storage/stable-diffusion/stable-diffusion-webui
2024-12-22 14:28:22,809 | DEBUG : Downloading Embedding https://civitai.com/api/download/models/253081 into /tmp/stable-diffusion-models/embeddings
Downloading: InnaNobodySD15.pt [100.00%] - 45.05 MB/s
2024-12-22 14:28:23,206 | INFO : Download completed. File saved as: /tmp/stable-diffusion-models/embeddings/InnaNobodySD15.pt
2024-12-22 14:28:23,207 | INFO : Downloaded in 0s
Downloading: analogMadness_v70.safetensors [100.00%] - 164.13 MB/s
2024-12-22 14:28:43,647 | INFO : Download completed. File saved as: /tmp/stable-diffusion-models/analogMadness_v70.safetensors
2024-12-22 14:28:43,648 | INFO : Downloaded in 20s
2024-12-22 14:28:43,649 | INFO : Removing broken symlinks...
2024-12-22 14:28:43,688 | INFO : Linking checkpoints...
2024-12-22 14:28:43,689 | INFO : Linking .ckpt and .safetensor/.safetensors/.st files in /tmp/stable-diffusion-models
2024-12-22 14:28:43,692 | INFO : /tmp/stable-diffusion-models/aZovyaPhotoreal_v3.safetensors 

In [8]:
%cd "{stable_diffusion_webui_path}"

# Code to set the options you want as defined in the very first block
x_arg = '--xformers' if activate_xformers else ''
dd_arg = '--deepdanbooru' if activate_deepdanbooru else ''
mvram_arg = '--medvram' if activate_medvram else ''
pickled = '--disable-safe-unpickle' if disable_pickle_check else ''
port = f'--port {gradio_port}' if gradio_port else '--share'
auth = f'--gradio-auth {gradio_auth} --enable-insecure-extension-access' if gradio_auth else ''
theme = f'--theme {ui_theme}' if ui_theme else ''
insecure_extension_access = '--enable-insecure-extension-access' if insecure_extension_access else ''
queue = '--gradio-queue' if gradio_queue else ''

# Launch args go below:
logger.info("Launching A1111 WebUI")
!python webui.py {x_arg} {dd_arg} {mvram_arg} {pickled} {port} {auth} {theme} {queue}

/storage/stable-diffusion/stable-diffusion-webui
2024-12-22 14:32:36,527 | INFO : Launching A1111 WebUI
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
no module 'xformers'. Processing without...
no module 'xformers'. Processing without...
No module 'xformers'. Proceeding without it.
*** Cannot import xformers
    Traceback (most recent call last):
      File "/storage/stable-diffusion/stable-diffusion-webui/modules/sd_hijack_optimizations.py", line 160, in <module>
        import xformers.ops
    ModuleNotFoundError: No module named 'xformers'

---
Loading weights [9356775e08] from /storage/stable-diffusion/stable-diffusion-webui/models/Stable-diffusion/aZovyaPhotoreal_v3.safetensors
fatal: No names found, cannot describe anything.
Running on local URL:  http://

## Nuke Machine

In [8]:
!rm -rf /storage/*
!mv /notebooks/*.ipynb / # move the notebook out of the directory before we nuke it
!rm -rf /notebooks/*
!mv /*.ipynb /notebooks/ # move it back
!rm -rf {model_storage_dir}
!rm -rf {repo_storage_dir}

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
